# ECR Top Up Analysis

## December 2017 Billing

Feb 13, 2018

In [1]:
import pandas as pd
from pandas import ExcelWriter
import numpy as np
from scipy import stats
import glob

In [9]:
#Import only the below columns from the Shortfall file:
column_names = ['GFCID','GFPID','GFPID Name','Customer Number','Customer Name','Billable Entity','Account Number',
                'Account Name','Bill Period','Account Category','Waive Flag','Account Type','Dep Assmt Eligible',
                'ECR Average Available Balance','Earnings Credit Rate %','Earnings Credits Earned','Gross Fees']

In [10]:
month = '01'
year = '2018'
path = year+month

In [11]:
#Read in data for US Balances and ECR:
US_Shortfall = pd.read_excel(path+"_US Shortfall.xlsx", skiprows=1, usecols=column_names)

In [12]:
#Import only the below columns from the Fee files (Billed Metrics):
billedMetricsColumns = ['REGION','BRANCH','ULTIMATE_PARENT_NAME','PRIMARY_CUSTOMER_NUMBER','PRIMARY_CUSTOMER_NAME',
                        'IGA_ACCOUNT','ACCOUNT_NUMBER','ACCOUNT_NAME','GFCID','GFPID','SERVICE_CODE',
                        'SERVICE_DESCRIPTION','CHARGE_CODE','CHARGE_CODE_DESCRIPTION','TRANSOUC','BILL_PERIOD',
                        'PRICE_METHOD','CHARGE_AMOUNT_IIC','INVOICE_CCY','TAX_AMOUNT_VAT_IIC','TAX_AMOUNT_STAMP_IIC']

In [14]:
#Import Fee data into separate dataframes to be merged later.
NAM_Fees = pd.read_excel(path+"_NAM Billed Metrics.xlsx", usecols=billedMetricsColumns)
Worldlink_Fees = pd.read_excel(path+"_WL Billed Metrics.xlsx", usecols=billedMetricsColumns)
Mexico_Fees = pd.read_excel(path+"_Mexico Billed Metrics.xlsx", usecols=billedMetricsColumns)


In [15]:
CEEMEA_Fees = pd.read_excel(path+"_UAE Billed Metrics.xlsx", usecols=billedMetricsColumns)
WEUR_Fees = pd.read_excel(path+"_WEUR Billed Metrics.xlsx", usecols=billedMetricsColumns)

In [16]:
#Get Expired Credits for NAM
NAM_ExpiredCredits = NAM_Fees[NAM_Fees.PRICE_METHOD.str.contains("EC Expiration") == True]

In [17]:
#Pivot Expiration by GFPID:
Expiration_Append = pd.pivot_table(NAM_ExpiredCredits,
                          index=["GFPID"],
                          values=["CHARGE_AMOUNT_IIC"],
                             aggfunc = np.sum)
Expiration_Append.reset_index(inplace=True)

Expiration_Append.head()

,GFPID,CHARGE_AMOUNT_IIC
0,2887,928.54
1,6769,1707.58
2,15067,86.67
3,19542,146.99
4,23191,2613.61


In [18]:
#Remove ECR transactions from Billed Metrics files
Worldlink_Fees = Worldlink_Fees[Worldlink_Fees.PRICE_METHOD.str.contains("EC") == False]
Worldlink_Fees = Worldlink_Fees[Worldlink_Fees.PRICE_METHOD.str.contains("Earn Cred") == False]

In [19]:
#Remove ECR transactions from Billed Metrics files
CEEMEA_Fees = CEEMEA_Fees[CEEMEA_Fees.PRICE_METHOD.str.contains("EC") == False]
CEEMEA_Fees = CEEMEA_Fees[CEEMEA_Fees.PRICE_METHOD.str.contains("Earn Cred") == False]

In [20]:
#Remove ECR transactions from Billed Metrics files
Mexico_Fees = Mexico_Fees[Mexico_Fees.PRICE_METHOD.str.contains("EC") == False]
Mexico_Fees = Mexico_Fees[Mexico_Fees.PRICE_METHOD.str.contains("Earn Cred") == False]

In [21]:
#Remove ECR transactions from Billed Metrics files
WEUR_Fees = WEUR_Fees[WEUR_Fees.PRICE_METHOD.str.contains("EC") == False]
WEUR_Fees = WEUR_Fees[WEUR_Fees.PRICE_METHOD.str.contains("Earn Cred") == False]

In [22]:
#Load Exchange Rates for USD equivalent calculations
dfRate=pd.read_excel("Exchange Rates.xlsx")

In [23]:
#Get unique Customer Numbers from Shortfall file to map GFPID. Will be used for WEUR Sharing GFPID Mapping 
Customer_GFPID = pd.pivot_table(US_Shortfall,
                          index=["Customer Number", "GFPID"],
                               values = ["Gross Fees"])
Customer_GFPID.reset_index(inplace=True)

In [24]:
Customer_GFPID = Customer_GFPID.drop(["Gross Fees"], axis=1)
Customer_GFPID.head()

,Customer Number,GFPID
0,10138,208817
1,11983,1001601233
2,14354,14354
3,19224,19224
4,24793,1001343056


In [26]:
#Load Sharing file
US_Sharing_To_WEUR = pd.read_excel(path+"_EC Sharing.xlsx", skiprows=1)

In [27]:
#Rename column for distinct value and filter for WEUR Sharing only:
US_Sharing_To_WEUR = US_Sharing_To_WEUR.rename(index=str, columns={"Region.1": "RegionTo"})
US_Sharing_To_WEUR = US_Sharing_To_WEUR[US_Sharing_To_WEUR.RegionTo.str.contains("Western Europe") == True]

In [28]:
Sharing_Append = pd.pivot_table(US_Sharing_To_WEUR,
                          index=["Main Customer ID"],
                          values=["USDE.1"],
                             aggfunc = np.sum)
Sharing_Append.reset_index(inplace=True)

In [29]:
Sharing_Append.head()

,Main Customer ID,USDE.1
0,1000089962,-1838.71
1,1000170921,-10000.00
2,1000370807,-1368.11
3,1001723223,-2752.28
4,1002258699,-855.38


In [30]:
#Merge GFPID to merge with fee data:
Sharing_Append_GFPID = pd.merge(Sharing_Append,Customer_GFPID, how='left', left_on='Main Customer ID', 
                                right_on='Customer Number')


In [31]:
#Pivot Sharing by GFPID, removing by customer:
Sharing_Append_GFPID = pd.pivot_table(Sharing_Append_GFPID,
                          index=["GFPID"],
                          values=["USDE.1"],
                             aggfunc = np.sum)
Sharing_Append_GFPID.reset_index(inplace=True)

Sharing_Append_GFPID.head()

,GFPID,USDE.1
0,2887,-52136.77
1,23191,-17888.69
2,105392,-1661.00
3,153257,-450.76
4,236098,-120.00


In [32]:
#Rename Column
Sharing_Append_GFPID = Sharing_Append_GFPID.rename(index=str, columns={"USDE.1": "US Sharing to WEUR"})
Sharing_Append_GFPID[(Sharing_Append_GFPID.GFPID == 130656)]

,GFPID,US Sharing to WEUR


In [35]:
#Renaming Columns that require filtering (can't have spaces)
US_Shortfall = US_Shortfall.rename(index=str, columns={"Account Type": "AccountType",
                                                       "Account Category": "AccountCategory" })

#Build pivot just for balances, ECR amounts and rates. Will bring in Total Fees for each region next
US_ECR = pd.pivot_table(US_Shortfall[(US_Shortfall.AccountType == "Balance") & 
                                     (US_Shortfall.AccountCategory == "Third Party")],
                          index=["GFPID"],
                          values=["ECR Average Available Balance","Earnings Credit Rate %","Earnings Credits Earned",
                                 "Gross Fees","Net Fees"],
                          aggfunc={"ECR Average Available Balance":np.sum,'Earnings Credit Rate %':stats.mode,
                                   'Earnings Credits Earned':np.sum})

US_ECR.reset_index(inplace=True) #Resetting the index so merge functions (VLOOKUPS) will work

In [36]:
US_Append = pd.pivot_table(US_Shortfall,
                          index=["GFPID"],
                          values=["Gross Fees"],
                             aggfunc = np.sum)

US_Append.reset_index(inplace=True)

In [37]:
WEUR_Fees = pd.merge(WEUR_Fees, dfRate, how='left', left_on=['INVOICE_CCY'], right_on=['CCY'])
WEUR_Fees["WEUR_Fees_USDE"] = WEUR_Fees["CHARGE_AMOUNT_IIC"] * WEUR_Fees["Rate"]

WEUR_Append = pd.pivot_table(WEUR_Fees,
                          index=["GFPID"],
                          values=["WEUR_Fees_USDE"],
                             aggfunc = np.sum)

WEUR_Append.reset_index(inplace=True)

In [38]:
Worldlink_Fees = pd.merge(Worldlink_Fees, dfRate, how='left', left_on=['INVOICE_CCY'], right_on=['CCY'])
Worldlink_Fees["Worldlink_Fees_USDE"] = Worldlink_Fees["CHARGE_AMOUNT_IIC"] * Worldlink_Fees["Rate"]

Worldlink_Append = pd.pivot_table(Worldlink_Fees,
                          index=["GFPID"],
                          values=["Worldlink_Fees_USDE"],
                             aggfunc = np.sum)

Worldlink_Append.reset_index(inplace=True)

In [39]:
CEEMEA_Fees = pd.merge(CEEMEA_Fees, dfRate, how='left', left_on=['INVOICE_CCY'], right_on=['CCY'])
CEEMEA_Fees["CEEMEA_Fees_USDE"] = CEEMEA_Fees["CHARGE_AMOUNT_IIC"] * CEEMEA_Fees["Rate"]

CEEMEA_Append = pd.pivot_table(CEEMEA_Fees,
                          index=["GFPID"],
                          values=["CEEMEA_Fees_USDE"],
                             aggfunc = np.sum)

CEEMEA_Append.reset_index(inplace=True)

In [40]:
Mexico_Fees = pd.merge(Mexico_Fees, dfRate, how='left', left_on=['INVOICE_CCY'], right_on=['CCY'])
Mexico_Fees["Mexico_Fees_USDE"] = Mexico_Fees["CHARGE_AMOUNT_IIC"] * Mexico_Fees["Rate"]

Mexico_Append = pd.pivot_table(Mexico_Fees,
                          index=["GFPID"],
                          values=["Mexico_Fees_USDE"],
                             aggfunc = np.sum)

Mexico_Append.reset_index(inplace=True)

In [41]:
result = pd.merge(US_ECR,US_Append, how='left', left_on='GFPID', right_on='GFPID')
result = pd.merge(result,Expiration_Append, how='left', left_on='GFPID', right_on='GFPID')
result = pd.merge(result,WEUR_Append, how='left', left_on='GFPID', right_on='GFPID')
result = pd.merge(result,Worldlink_Append, how='left', left_on='GFPID', right_on='GFPID')
result = pd.merge(result,CEEMEA_Append, how='left', left_on='GFPID', right_on='GFPID')
result = pd.merge(result,Mexico_Append, how='left', left_on='GFPID', right_on='GFPID')
result = pd.merge(result,Sharing_Append_GFPID, how='left', left_on='GFPID', right_on='GFPID')

In [42]:
result = result.rename(index=str, columns={"Gross Fees": "US Fees_USD"})

In [43]:
result = result.rename(index=str, columns={"CHARGE_AMOUNT_IIC": "Expired Credits"})

In [44]:
result = result.fillna(0)

In [31]:
#result["Shortfall / (Longfall)"] = result["Earnings Credits Earned"]+result["US Fees_USD"]-result["US Sharing to WEUR"]

In [45]:
result.head()

,GFPID,ECR Average Available Balance,Earnings Credit Rate %,Earnings Credits Earned,US Fees_USD,Expired Credits,WEUR_Fees_USDE,Worldlink_Fees_USDE,CEEMEA_Fees_USDE,Mexico_Fees_USDE,US Sharing to WEUR
0,0,5079128,"([0.0007], [57])",-138.53,439455.89,0.00,18.607500,0.000000,0.000000,0,0.00
1,1864,5416,"([0.0025], [1])",-1.15,292.55,0.00,0.000000,0.000000,0.000000,0,0.00
2,2887,694448013,"([0.0125], [622])",-737256.50,642182.35,928.54,5661.879691,194054.105384,50084.840647,0,-52136.77
3,4197,0,"([0.0], [1])",0.00,0.00,0.00,0.000000,0.000000,0.000000,0,0.00
4,6769,115094602,"([0.0035], [21])",-34213.06,38361.15,1707.58,17699.818127,121.600000,0.000000,0,0.00


In [46]:
writer = ExcelWriter(path+'_Consolidated Fees by GFPID.xlsx')
result.to_excel(writer,'ECR Top Up', index=False)
writer.save()

In [36]:

#for i, row in Worldlink_Append.iterrows():
#    Worldlink_Append.iloc[i,0]= Worldlink_Append.iloc[i,0].lstrip("0")
#Worldlink_Append1 = Worldlink_Append.set_index("GFPID")